# botTimeline
Bot Twitter que simula usuário vend tweets mensagens periodicamente. Esses tweets são capturados e salvos em um banco Mongo.

Este notebook é um guia! Você vai precisar rodar localmente o código, mas não pule essa etapa, pois é muito importante entender como funciona a ferramenta e como manipular os dados que você coletar.

# Configurando o Ambiente
Pré-requisitos básicos:
  1. Python3.x, pip3 e mongoDB instalados.
  2. Clone o projeto em https://github.com/tcruzfranca/botTimeline.
  3. Baixe o Driver do navegador Chrome(verifique sua versão) em: http://chromedriver.chromium.org/downloads.
  4. Faça uma cópia do ChromeDrive na mesma pasta onde está o projeto
  5. Instale ou atualize o seu Google Chrome para a versão mais recente.
  6. Instale as dependências (se estiver no Linux, você pode precisar instalar como superusuário): 
      
    pip3 install requirements.txt

# Criando um Bot

Será necessário criar um arquivo de configuração com parâmetros básicos para o bot (veja o arquivo de exemplo "botParams.txt"):
          #timeoutNavegador in seconds
          timeoutNavegador = 30

          #timeToLogIn in seconds
          timeToLogIn = 3

          user = "seu_usuario_no_twitter"
          password = "senha_usuario_twitter"

          #timeToLoadTheTimeLine in seconds
          timeToLoadTheTimeLine = 3

          #Mongo params
          db = "nnome_banco_mongo"
          collection = "colecao_mongo"

          #amountOfTweets max 35 a 38
          amountOfTweets = 20

          #intervalo entre coletas em segundos (1200sec, ou seja, 20 min no exemplo)
          botSleepTime = 1200


O arquivo com as configurações do bot precisa ter os seguintes campos obrigatórios: timeoutNavegador, timeToLogIn, user, timeToLoadTheTimeLine, db, collection, amountOfTweets e botSleepTime. Não existem parâmetros opcionais.
Linhas que iniciam "#" são comentários. Não serão lidas pelo arquivo de manipulação do bot.
As linhas em branco serão ignoradas.

Depois rode o comando:
      
      python3 timelineScrap.py.

Ao rodar o arquivo você precisará fornecer o [path/arquivo_config_bot] do arquivo que contém as configurações do bot. Veja o exemplo abaixo de como será a mensagem apresentada:
      
      Arquivo de configuração com o path (ex: path/settings.txt): [path/arquivo_config_bot]

O processo timelineScrap.py irá permanecer rodando até que seja interrompido pelo usuário.

# Definindo um tweet


   O primeiro passo a ser tomado é criar uma classe com todos os atributos de um Tweet que podem ser interessantes para qualquer tipo de análise, como a quantidade de likes ou retweets que uma publicação(Tweet) recebeu, a hora de envio, um código de identificação associado a esse Tweet e seu conteúdo.

In [2]:
'''
__author__ = "Tiago Cruz de França"
__copyright__ = "Copyright 2018, UFRRJ"
__credits__ = ["Tiago França"]
__license__ = "GPL"
__version__ = "0.0.1"
__maintainer__ = "Tiago França, Edu Mangabeira"
__email__ = "tcruz.franca@gmail.com, edu.mangaba@gmail.com"
__status__ = "Test"

'''

'\n__author__ = "Tiago Cruz de França"\n__copyright__ = "Copyright 2018, UFRRJ"\n__credits__ = ["Tiago França"]\n__license__ = "GPL"\n__version__ = "0.0.1"\n__maintainer__ = "Tiago França, Edu Mangabeira"\n__email__ = "tcruz.franca@gmail.com, edu.mangaba@gmail.com"\n__status__ = "Test"\n\n'

In [3]:
class Tweet(object):
    
    def __init__(self, id_tweet,text,amount_of_comments,amount_of_likes, amount_of_retweets,created_at,screen_name, publisher_id, publisher_screen_name,snapshot_id,snapshot_timestamp,impression_order):
        '''
            @attributes:
                id_tweet: The tweet's Id.
                tweet_text: The catched up text.
                retweet_count: The amout of times the tweet was republished.
                tweet_created_at: The data when the message was published.                
                bot_screen_name: The user's Twitter name. E.g. @botPalestrinha
                publisher_screen_name: the author of the tweet.                                   
        
        '''
        self.id_tweet = id_tweet
        self.tweet_text = text
        self.tweet_amount_of_comments = amount_of_comments#
        self.tweet_amount_of_likes = amount_of_likes
        self.retweet_count = amount_of_retweets
        self.tweet_created_at = created_at
        self.bot_screen_name = screen_name
        self.publisher_id = publisher_id
        self.publisher_screen_name = publisher_screen_name
        self.snapshot_id = snapshot_id
        self.snapshot_timestamp = snapshot_timestamp
        self.impression_order = impression_order

# Conhecendo as funcionalidades do bot

Vamos começar importando as bibliotecas, que serão explicadas mais à frente.

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from Tweet import Tweet
import json
import time
import re
import pymongo
import datetime

ModuleNotFoundError: No module named 'pymongo'

A função abaixo é a de extração textual. Salva o texto do nosso tweet descartando caracteres irrelevantes usando expressões regulares(bilioteca 're') e removendo espaços em branco no início e/ou final do tweet com strip() 

In [5]:
def extraindoApenasTexto(tweet_texto):
    
    texto = re.sub('<[^>]*>'," ",tweet_texto)
    texto = re.sub('\s+',' ',texto)
    texto = texto.strip()
    return texto

A partir daqui será importante o uso do webdriver, que irá abrir um navegador de forma isolada, sem que dados(cookies, histórico) da sua conta google padrão, por exemplo, interfiram no comportamento do seu bot.

In [8]:
def getDriverOnTwitter(timeout):
    # cria uma nova sessão isolada
    driver = webdriver.Chrome("./chromedriver")
    driver.implicitly_wait(timeout)#30 segundos para iniciar o browser
    driver.maximize_window()#maximiza tela ao iniciar
    # Navega até a página inicial do twitter
    driver.get("https://twitter.com")    
    return driver

Agora com um browser em execução, é necessário fazer login com a conta do nosso bot. Usaremos o Selenium
para procurar elementos no código fonte da página e interagir com eles, tanto para o login quanto para
as próximas etapas que envolvem coletar os dados de um tweet presente na timeline de nosso bot.

In [10]:
def fazerLogin(usuario,senha,driver):

    user = driver.find_element_by_name('session[username_or_email]')
    user.clear()
    user.send_keys(usuario)

    password = driver.find_element_by_name('session[password]')
    password.clear()
    password.send_keys(senha)

    password.submit()#Simula o uso do botão enter após entrar com a senha


O screen_name é o username de um usuário. exemplo: "@bot235".

In [ ]:
def getBotScreenName(driver):
    # currently on result page using find_elements_by_class_name method
    lists = driver.find_elements_by_class_name('u-linkComplex-target')

    bot_screen_name = ""
    for listitem in lists:
        if (listitem.get_attribute("class") == "u-linkComplex-target"):
            bot_screen_name = listitem.get_attribute("innerHTML")
            
            break
    return bot_screen_name

A próxima função foi feita para seguir automaticamente contas a serem predefinidas no arquivo "sources.txt", você precisa inserir o screen_name das contas que deseja seguir no Twitter.

Atenção!!! O uso dessa função é recomendado apenas caso você ainda não tenha seguido todas as fontes, ela percorre toda a lista de nomes, então é aconselhável que você use apenas na primeira vez que for executar e depois a desative. Você pode fazer isso comentando todo o bloco ou escrevendo a diretiva "pass" no final da função.

Pode levar algum tempo para seguir todas as fontes, visto que existe um timer(time.sleep()) para controlar o acesso
e reduzir a chance do bot ser derrubado pelo twitter. Tenha paciência e não extrapole o limite de requisições da plataforma.

Outro ponto importante:e m vez de usar um "número mágico" como 48(Era o número de fontes que eu segui para uma pesquisa), você pode definir o seu próprio limite de quantos usuários o bot vai seguir.

In [13]:
def FollowsUser(driver):

    # follow_bound = seu_numero_aqui
    follow_list = {}
    follow_arc = open("FollowCount.txt",'w')
    for linha in follow_arc:
            texto = re.sub('#+[\s\S]*',"",linha)
            texto = texto.strip()
            if len(texto) > 0:
                
                key,value = texto.split("=")
                key = key.strip()
                value = int(value.strip())

                parametros[key]=value

            if(value < 48): #substituir 48 por follow_bound
                users_list = open('sources.txt','r')
                for user in users_list:
                    user = user.replace("\n","")
                    driver.get("https://twitter.com/%s" % user)
                    time.sleep(5)
                    try:
                        follow_button = driver.find_element(By.CSS_SELECTOR,"button.EdgeButton.EdgeButton--secondary.EdgeButton--medium.button-text.follow-text")
                        follow_button.click()
                        value += 1
                    except:
                        if (follows == 48): #substituir 48 por follow_bound
                            print("Already follows all") #indica que todas as contas foram seguidas
                            break
                            time.sleep(10)
                    follow_arc.close()
    follow_list.close()
    #pass

Lembra que definimos uma classe com os atributos de um tweet? É preciso coletar no código fonte de um tweet todas essas informações importantes antes de associá-las à classe. Os atributos são coletados pelo Selenium e a função que definimos anteriormente(extraindoApenasTexto(tweet_texto)) vai limpar o texto do tweet.

In [ ]:
def criandoTweet(li,order,attributes_to_wrapped_tweet=[]):#eu pego tudo a partir de um element li

    _id = li.get_attribute("data-item-id")
    text = li.find_element(By.CSS_SELECTOR,"p.TweetTextSize.js-tweet-text.tweet-text").get_attribute("innerHTML")        
    text = extraindoApenasTexto(text)

    amount_of_comments = li.find_element_by_class_name("ProfileTweet-actionCountForPresentation").get_attribute("innerHTML")
    amount_of_likes = li.find_element_by_class_name("ProfileTweet-actionCountForPresentation").get_attribute("innerHTML")
    amount_of_retweets = li.find_element_by_class_name("ProfileTweet-actionCountForPresentation").get_attribute("innerHTML")
    timestamp = li.find_element(By.CSS_SELECTOR,"a.tweet-timestamp.js-permalink.js-nav.js-tooltip").get_attribute("title")

    publisher_id = li.find_element(By.CSS_SELECTOR,"a.account-group.js-account-group.js-action-profile.js-user-profile-link.js-nav").get_attribute("data-user-id")
    publisher_name = li.find_element(By.CSS_SELECTOR, "span.username.u-dir.u-textTruncate").find_element_by_tag_name("b").get_attribute("innerHTML")

    bot_screen_name = attributes_to_wrapped_tweet[0]
    snapshot_id = attributes_to_wrapped_tweet[1]
    snapshot_timestamp = attributes_to_wrapped_tweet[2]
    impression_order = order

    tweet = Tweet(_id,text,amount_of_comments,amount_of_likes,amount_of_retweets,timestamp,bot_screen_name,publisher_id,publisher_name,snapshot_id,snapshot_timestamp,impression_order)

    return tweet

In [ ]:
def pegandoTweets(tweets_lis,collection,numTweets=20,attributes_to_wrapped_tweet=[]):

    tweets = []
    order = 0
    for li in tweets_lis:
        try:
            tweet = criandoTweet(li,order,attributes_to_wrapped_tweet)

            serializedTweet= json.dumps(tweet.__dict__)            
            serializedTweet = json.loads(serializedTweet)

            collection.insert_one(serializedTweet)

            order += 1
            
            if (order == numTweets):
                break
              
        except:
            print ("excecao em pegandoTweets")

Carrega os últimos tweets na pilha da timeline. Vai nos ajudar a pegar os tweets fornecendo o parâmetro tweet_lis, da função acima.

In [ ]:
def pegandoTimeline(driver):
    timeline = driver.find_elements(By.CSS_SELECTOR,".stream")#find_elements_by_class_name pode usar tb
    element = timeline[0]
    tweets_lis = element.find_elements(By.CSS_SELECTOR, "li.js-stream-item.stream-item.stream-item")#ou 
    return tweets_lis

A função abaixo vai nos conectar ao banco de dados mongo e permitir o registro dos dados coletados(ainda chegaremos nessa etapa). O seu retorno será uma coleção a ser usada de parâmetro da função pegandoTweets.

Agora vai uma explicação rápida sobre o banco mongo, você pode pular essa parte por enquanto, pois só vai poder testar depois que todo o código for executado e os dados obtidos.

1) Para iniciar no seu terminal, basta digitar 'mongo'.

2) Primeiro é preciso ver quais bancos estão sendo usados na sua máquina, o comando a ser usado é 'show dbs'.

3) Para usar um banco: 'use {insira aqui um banco, sem chaves}'

4) Agora temos que selecionar uma coleção do nosso banco, para ver as coleções: 'show collections'.

5) Existem diferentes formas de acessar os dados, vou citar algumas.

 5.1) 'db.{sua collection}.find()' mostra alguns tweets, é útil para verificar rapidamente erros em parâmetros. Também é possível filtrar a procura, para saber com mais detalhes veja em: https://docs.mongodb.com/manual/reference/method/db.collection.find/
 
 5.2) 'db.{sua collection}.findOne()' mostra apenas um tweet e também pode filtrar a busca(recomendado nesse caso).
    
Se você deseja apenas um arquivo com a base de dados, volte para o seu terminal(crtl+C interrompe o processo mongo) e digite o seguinte comando:

#### Para obter .json

mongoexport --db {seu_banco} --collection {sua_coleção} --out {diretorio/nome_do_seu_arquivo}.json

#### Para obter .csv

mongoexport --db {seu_banco} --collection {sua_coleção} --type=csv --out {diretorio/nome_do_seu_arquivo}.csv

Existem opções mais específicas não citadas aqui, como obter apenas alguns campos da tabela. Aqui está a documentação do MongoDB, caso queria usar mais alguma função não apresentada aqui.
https://docs.mongodb.com/manual/tutorial/getting-started/

In [14]:
def getMongoCollection(db,col):
    mongo = pymongo.MongoClient()
    db = mongo[db]
    col = db[col]
    return col

O snapshot marca o momento em que o dado foi coletado. Pode ser útil cruzar essa informação com a data de publicação de um tweet, por exemplo.

In [15]:
def obterLastSnapshotId(collection):
    try:
        register = collection.find({},{"_id":0,"snapshot_id":1}).sort("snapshot_id",pymongo.DESCENDING).limit(1)
        register = register.next()
        resultado = register["snapshot_id"]
    except:
        print ("except snapshot_id")
        resultado = 0

    return resultado

In [1]:
# lê o arquivo com as suas configurações
def getParametrosBot(arquivo):

    parametros = {}
    for linha in arquivo:
        texto = re.sub('#+[\s\S]*',"",linha)
        texto = texto.strip()
        if len(texto) > 0:
            
            chave,valor = texto.split("=")
            chave = chave.strip()
            valor = valor.strip()
            
            parametros[chave]=valor

    return parametros

# Coletador

#### Atenção! só rode essa função se você quiser que um ÚNICO bot abra UM navegador.

In [ ]:
def rodarColetador(arquivo):

    arquivo = open(arquivo)
    parametros = getParametrosBot(arquivo)

    collection = getMongoCollection(parametros['db'],parametros['collection'])

    #verificando se existem outros ids no banco de coletas anteriores
    snapshot_id = obterLastSnapshotId(collection)
        
    driver = getDriverOnTwitter(parametros['timeoutNavegador'])
    time.sleep(int(parametros['timeToLogIn']))
    user = parametros['user']
    password = parametros['password']
    fazerLogin(user,password,driver)
    #waiting the user login before catch the desired fields
    time.sleep(int(parametros['timeToLoadTheTimeLine']))
    bot_screen_name = getBotScreenName(driver)

    #bot_follow_count = parametros['botFollowCount']
    #verifies whether the bot follows all users or not
    #FollowsUser(driver)

    while True:

        tweets_lis = pegandoTimeline(driver)
        
        snapshot_id += 1
        snapshot_timestamp = datetime.datetime.utcnow().strftime("%a %b %d %H:%M:%S +0000 %Y")
        
        attributes_to_wrapped_tweet = [bot_screen_name,snapshot_id,snapshot_timestamp]
        
        pegandoTweets(tweets_lis,collection,int(parametros['amountOfTweets']),attributes_to_wrapped_tweet)
        
        #colocando o bot para dormir
        time.sleep(int(parametros['botSleepTime']))
        #
        try:
            print("atualiza?")
            driver.find_element(By.CSS_SELECTOR,"button.new-tweets-bar.js-new-tweets-bar").click()
            print ("atualizou")
        except:
            continue

if __name__ == '__main__':

    arquivo = input("Arquivo de configuração com o path (ex: path/settings.txt):")
    rodarColetador(arquivo)

# Rodando Múltiplos Bots em Processos Paralelos
Também é possível rodar vários bots. Cada um deles irá abrir um novo navegador em sessões distintas (isoladas). Dessa forma, você pode rodar diferentes bots paralelamente. Os bots criados também serão associados a processos distintos. Você deve ter cuidado com a memória consumida por cada processo individual e por todos juntos (a soma do uso dos recursos individualmente). Esses processos podem usar todos os núcleos do seu processador (o sistema operacional irá controlar isso), pois não foi incluída nenhuma restrição no script de paralelização dos bots.

Rodando múltiplos bots como processos autônomos:
  1. Crie todos os arquivos de configuração conforme descrito em "Criando um Bot". Cada bot precisa ter um arquivo distinto (veja o exemplo "botParams.txt").
  2. Em um novo arquivo, adicione os arquivos com as configuração dos bots (um por linha). Por exemplo, considere que existem dois arquivos de configuração de bot (botParams1.txt e botParams2.txt). O arquivo para múltiplos bots deve informar cada arquivo de configuração (um por linha) como segue (veja o exemplo "listaArquivos.txt"):
  
          path/testeParms.txt
          path/testeParms2.txt
  
  3. Rode o comando multTimelineScrap.py.
          
          python3 multTimelineScrap.py
  
  4. Você precisará informar o arquivo que aponta para todas as configurações de bot.
       
          Arquivo com lista de configuracoes:[path/botsSettings.txt]

O script irá criar um scraper bot para cada arquivo de configuração de bot listado. Ou seja, se você informar "1", será um bot. Se informar "2", serão dois. Se informar "10", serão dez bots, etc. Os bots permanecerão rodando até que o seu processo associado ao multTimelineScrap.py sejá interrompido pelo usuário (ou o navegador aberto pelos bots seja fechado).


In [ ]:
from multiprocessing import Pool
import timelineScrap

__author__ = "Tiago Cruz de França"
__copyright__ = "Copyright 2018, UFRRJ"
__credits__ = ["Tiago França"]
__license__ = "GPL"
__version__ = "0.0.1"
__maintainer__ = "Tiago França, Edu Mangabeira"
__email__ = "tcruz.franca@gmail.com"
__status__ = "Test"

if __name__ == '__main__':

    listaArquivos = input("Arquivo com lista de configuracoes:")
    listaArquivos = open(listaArquivos)
    #resp = input("Deseja seguir mais usuários do Twitter?  s - sim  n - não")
    #cont = timelineScrap.getFollowIsFalse(resp)

    
    arquivos = []
    for arquivo in listaArquivos:
        arquivo = arquivo.replace("\n","")
        #arquivo = open(arquivo)
        arquivos.append(arquivo)
    
    with Pool() as p:
        p.map(timelineScrap.rodarColetador,arquivos)    
    print ("rodando")

# Carregando o dataset

A partir deste ponto você pode seguir como quiser, fica aqui apenas uma sugestão de como carregar o dataset.

Se você é minimamente familiar com a linguagem R, abra o Rstudio e digite no terminal dele os comandos abaixo.

> install.packages("jsonlite")

> library(jsonlite)

> nome_do_arquivo <- '/SeuLocalAqui/seuArquivo.json'

> dataset <-jsonlite::stream_in(textConnection(readLines(nome_do_arquivo, n=1000)),verbose=F)

Onde na última linha(dataset) o argumento "n=1000" do método readLines() se refere ao número máximo de linhas que você deseja visualizar, o que pode ser útil caso o dataset seja muito pesado e você não pretenda carregar tudo de uma vez.